# Libraries installation on the current python environment

# Pre-processing original data

In [1]:
import csv
def custom_header_reader(file:'TextIOWrapper'):
    csvReader = csv.reader(file,delimiter='\t')
    for row, text in enumerate(csvReader):
        if row == 2: numMarkers = int(text[-1])
        elif row == 10: 
            columnNames = text[:-1]
            if columnNames[0] == 'Frame': 
                break
        elif row == 11: columnNames = text[:-1]; break
    return numMarkers, columnNames, csvReader

def line_reader(csvReader,fromSecond,toSecond):
    for line in csvReader:
        if fromSecond <= float(line[1]) <= toSecond:
            yield line[:len(columnNames)]
        elif float(line[1]) > toSecond:
            break

In [2]:
sampleAnno = -1

## Reading from dataset

In [3]:
import os
import pandas as pd
ANNOTATIONS_PATH = 'data'
RAW_PATH = ANNOTATIONS_PATH+'/raw'
#with open(ANNOTATIONS_PATH+'/annotations.txt','r') as file:
#    annotations = file.read().splitlines()
#annotations_file = ANNOTATIONS_PATH+'/annotationsVExamples.txt' #'/annotationsVSingle.txt'
annotations_file = ANNOTATIONS_PATH+'/annotationsVExamples.txt' #'/annotationsVSingle.txt'
#annotations_file = ANNOTATIONS_PATH+'/annotationsVSingle.txt'
with open(annotations_file,'r') as file:
    annotations = file.read().splitlines()[1:]#[line for line in file.read().splitlines() if 'no val' in line]
    print(len(annotations))

if '(A)' in annotations[sampleAnno] or '(B)' in annotations[sampleAnno]:
    folder, trial, fragId, side, OoM, startSec, endSec = annotations[sampleAnno].replace(' ','').split(',')[:7]
else:
    folder, trial, fragId, OoM, startSec, endSec = annotations[sampleAnno].replace(' ','').split(',')[:6]
sampleData = os.path.join(RAW_PATH,folder,trial+'.tsv')
sampleData = os.path.join(ANNOTATIONS_PATH,folder,trial+'.tsv')
print(sampleData,startSec,endSec)
with open(sampleData,'r') as file:
    numMarkers, columnNames, readerBuffer = custom_header_reader(file)
    data = pd.DataFrame(line_reader(readerBuffer,float(startSec),float(endSec)),columns=columnNames).astype(dict(zip(columnNames,[int,float,str]+[float]*len(columnNames[3:]))))
if data['Time'].iloc[0] > float(startSec) or data['Time'].iloc[-1] < float(endSec):
    raise Exception(f'\x1b[31mNOT FULL FRAMES!!! data starts at: {data["Time"].iloc[0]} of {float(startSec)} and ends at {data["Time"].iloc[-1]} of {float(endSec)}\x1b[0m')
else:
    print("\x1b[32mSTART AND END SECONDS CHECK CORRECT!\x1b[0m")
data.head()

7
data/examples_for_thesis/t_044.tsv 1.19 36.55
START AND END SECONDS CHECK CORRECT!


,Frame,Time,SMPTE,ARIEL X,ARIEL Y,ARIEL Z,LFHD X,LFHD Y,LFHD Z,RFHD X,...,RBHD Z,LBHD X,LBHD Y,LBHD Z,RPLM X,RPLM Y,RPLM Z,LPLM X,LPLM Y,LPLM Z
0,120,1.19,05:53:33:20,-68.661,1717.567,-862.432,-133.771,1578.963,-900.647,-7.429,...,-735.169,-154.571,1658.581,-755.399,171.774,779.411,-623.616,-342.167,794.310,-578.771
1,121,1.20,05:53:33:20,-69.503,1715.807,-864.115,-134.528,1577.063,-901.216,-8.202,...,-736.168,-155.502,1658.017,-756.708,170.689,779.580,-623.943,-344.302,794.314,-579.032
2,122,1.21,05:53:33:20,-70.240,1714.043,-866.006,-135.303,1574.917,-901.878,-9.013,...,-737.423,-156.465,1657.275,-758.210,169.480,779.819,-623.015,-346.282,794.248,-579.592
3,123,1.22,05:53:33:20,-70.964,1712.050,-868.015,-136.033,1572.581,-902.626,-9.751,...,-738.771,-157.514,1656.321,-759.787,168.179,780.135,-621.637,-348.277,794.242,-580.390
4,124,1.23,05:53:33:21,-71.747,1709.951,-870.138,-136.821,1570.100,-903.444,-10.643,...,-740.181,-158.574,1655.262,-761.478,166.699,780.291,-620.096,-350.184,794.224,-581.416


## Standardizing column names

In [4]:
print(len(data))
data.iloc[:,3::3].head()

3537


,ARIEL X,LFHD X,RFHD X,C7 X,T5 X,T10 X,BWT X,FWT X,CLAV X,STRN X,...,RPNKY X,LTHMB X,LINDX X,LPNKY X,LFRM X,LFSH X,RBHD X,LBHD X,RPLM X,LPLM X
0,-68.661,-133.771,-7.429,-80.804,-94.567,-89.099,-93.199,-81.755,-87.373,-85.150,...,163.430,-264.447,-306.799,-320.366,-285.566,-172.836,-12.660,-154.571,171.774,-342.167
1,-69.503,-134.528,-8.202,-82.512,-96.204,-90.518,-94.290,-82.691,-88.242,-85.911,...,162.354,-265.706,-307.758,-322.696,-287.505,-173.931,-13.632,-155.502,170.689,-344.302
2,-70.240,-135.303,-9.013,-84.095,-97.838,-91.889,-95.286,-83.455,-89.205,-86.906,...,160.414,-266.978,-308.897,-324.952,-289.430,-174.783,-14.635,-156.465,169.480,-346.282
3,-70.964,-136.033,-9.751,-85.613,-98.965,-93.117,-96.253,-84.533,-90.156,-87.591,...,159.176,-268.364,-309.624,-327.130,-291.743,-175.122,-15.692,-157.514,168.179,-348.277
4,-71.747,-136.821,-10.643,-86.964,-100.269,-94.230,-97.195,-85.481,-91.008,-88.639,...,156.800,-269.591,-311.037,-329.238,-293.631,-175.952,-16.786,-158.574,166.699,-350.184


In [5]:
[colname[:-2] for colname in list(data.iloc[:1,3::3].columns)]

['ARIEL',
 'LFHD',
 'RFHD',
 'C7',
 'T5',
 'T10',
 'BWT',
 'FWT',
 'CLAV',
 'STRN',
 'LBSH',
 'LSHO',
 'LFUPA',
 'LBUPA',
 'LIEL',
 'LELB',
 'LOWR',
 'LIWR',
 'RSHO',
 'RBSH',
 'RFUPA',
 'RBUPA',
 'RFSH',
 'RIEL',
 'RELB',
 'RFRM',
 'ROWR',
 'RIWR',
 'LFWT',
 'RFWT',
 'LBWT',
 'RBWT',
 'LFTHI',
 'LBTHI',
 'LKNE',
 'LKNI',
 'LSHN',
 'LANK',
 'LHEL',
 'LTOE',
 'LMT1',
 'LMT5',
 'RFTHI',
 'RBTHI',
 'RKNE',
 'RKNI',
 'RSHN',
 'RANK',
 'RHEL',
 'RTOE',
 'RMT1',
 'RMT5',
 'RTHMB',
 'RINDX',
 'RPNKY',
 'LTHMB',
 'LINDX',
 'LPNKY',
 'LFRM',
 'LFSH',
 'RBHD',
 'LBHD',
 'RPLM',
 'LPLM']

In [6]:
import pandas as pd
if '_A' in data.columns[3] or '_B' in data.columns[3]:
    data = data[['Frame','Time','SMPTE']+data.filter(like=f'_{side[1]}').columns.tolist()]
    data.columns = [col.replace(f'_{side[1]}','') for col in data.columns]
elif '61A' in data.columns[3] or '61B' in data.columns[3]:
    data = data[['Frame','Time','SMPTE']+data.filter(like=f'61{side[1]}').columns.tolist()]
    data.columns = [col.replace(f'61{side[1]}_','') for col in data.columns]
if '_01' in data.columns[3]:
    data.columns = [column for column in data.columns[:3]]+[column[:-5]+column[-2:] for column in data.columns[3:]]
if 'HeadFront' in data.columns[3] or 'RKneeIn' in data.columns[3]:
    mappings = pd.read_csv("data/raw/Sport42MarkersTo64.csv")
    mappings = mappings.set_index(mappings.columns[0])[mappings.columns[1]].to_dict()
    data.columns = [mappings[col[:-2]]+col[-2:] if col[:-2] in mappings.keys() else col for col in data.columns]
    

In [7]:
list(data.columns)[:3]+list(data.columns)[3::3]

['Frame',
 'Time',
 'SMPTE',
 'ARIEL X',
 'LFHD X',
 'RFHD X',
 'C7 X',
 'T5 X',
 'T10 X',
 'BWT X',
 'FWT X',
 'CLAV X',
 'STRN X',
 'LBSH X',
 'LSHO X',
 'LFUPA X',
 'LBUPA X',
 'LIEL X',
 'LELB X',
 'LOWR X',
 'LIWR X',
 'RSHO X',
 'RBSH X',
 'RFUPA X',
 'RBUPA X',
 'RFSH X',
 'RIEL X',
 'RELB X',
 'RFRM X',
 'ROWR X',
 'RIWR X',
 'LFWT X',
 'RFWT X',
 'LBWT X',
 'RBWT X',
 'LFTHI X',
 'LBTHI X',
 'LKNE X',
 'LKNI X',
 'LSHN X',
 'LANK X',
 'LHEL X',
 'LTOE X',
 'LMT1 X',
 'LMT5 X',
 'RFTHI X',
 'RBTHI X',
 'RKNE X',
 'RKNI X',
 'RSHN X',
 'RANK X',
 'RHEL X',
 'RTOE X',
 'RMT1 X',
 'RMT5 X',
 'RTHMB X',
 'RINDX X',
 'RPNKY X',
 'LTHMB X',
 'LINDX X',
 'LPNKY X',
 'LFRM X',
 'LFSH X',
 'RBHD X',
 'LBHD X',
 'RPLM X',
 'LPLM X']

## Working on NaNs

In [8]:
print("Total NaNs")
nans = data[data.columns[(data.isna().sum(axis=0) > 0 ).to_list()]].isna().sum(axis=0).sort_values(ascending=False)
color = '\x1b[32m' if len(nans) == 0 else '\x1b[31m'
print(color+str(nans)+'\x1b[0m '+ ('\nNo Nans' if len(nans)==0 else '\nNans found!'))

Total NaNs
Series([], dtype: float64) 
No Nans


In [9]:
from itertools import groupby
max_nans_per_column = data.apply(lambda col: max(len(list(group))*value for value, group in groupby(col.isna().tolist())))
print("Max contiguous NaNs")
max_nans_per_column = max_nans_per_column[max_nans_per_column > 0].sort_values(ascending=False)
color = '\x1b[32m' if len(max_nans_per_column) == 0 else '\x1b[31m'
print(color+str(max_nans_per_column)+'\x1b[0m ' + ('\nNo Nans' if len(nans)==0 else '\nNans found!'))

Max contiguous NaNs
Series([], dtype: int64) 
No Nans


## Working on Zeros

In [10]:
print('Num Zeros')
zeros = data[data.columns[(data.eq(0.0).sum(axis=0) > 0).to_list()]].eq(0.0).sum(axis=0).sort_values(ascending=False)
color = '\x1b[32m' if len(zeros) == 0 else '\x1b[31m'
print(color+str(zeros)+'\x1b[0m ' + ('\nNo Zeros' if len(zeros)==0 else '\nZeros found!'))
#list(zeros.index)

Num Zeros
Series([], dtype: float64) 
No Zeros


In [11]:
import numpy as np
#for name_col,num_zeros in zeros.items():
#    if num_zeros < 20:
#        data[name_col] = data[name_col].replace(0.0,np.nan).interpolate(method='spline', order=3)

# THIS IS A FIX FOR MISSING QTM AT DATA 58-59
for name_col,num_zeros in zeros.items():
    if num_zeros < 20 and name_col != 'Time' and False:
        data[name_col] = data[name_col].replace(0.0,np.nan).interpolate(method='spline', order=3)

In [12]:
print('Num Zeros')
zeros = data[data.columns[(data.eq(0.0).sum(axis=0) > 0).to_list()]].eq(0.0).sum(axis=0).sort_values(ascending=False)
color = '\x1b[32m' if len(zeros) == 0 else '\x1b[31m'
print(color+str(zeros)+'\x1b[0m ' + ('\nNo Zeros' if len(zeros)==0 else '\nZeros found!'))
#list(zeros.index)

Num Zeros
Series([], dtype: float64) 
No Zeros


In [13]:
from itertools import groupby
max_zeros_per_column = data.apply(lambda col: max(len(list(group))*value for value, group in groupby(col.eq(0.0).tolist())))
print("Max contiguous Zeros")
max_zeros_per_column = max_zeros_per_column[max_zeros_per_column > 0].sort_values(ascending=False)
color = '\x1b[32m' if len(max_zeros_per_column) == 0 else '\x1b[31m'
print(color+str(max_zeros_per_column)+'\x1b[0m ' + ('\nNo Zeros' if len(max_zeros_per_column)==0 else '\nZeros found!'))

Max contiguous Zeros
Series([], dtype: int64) 
No Zeros


## Converting to reduced marker set

![fullbodymarkers](../resources/fullmarkerset.png) ![markerstable](../resources/markersetable.png)

nono smarmella e apri tutto

In [14]:
def map_reduce_num_markers(reducedMarkerNames:list):
    markersMap = {reducedMarkerNames[0]:   ['RHEL','RMT1','RMT5','RTOE'],
                  reducedMarkerNames[1]:   ['LHEL','LMT1','LMT5','LTOE'],
                  reducedMarkerNames[2]:   ['RANK'],
                  reducedMarkerNames[3]:   ['LANK'],
                  reducedMarkerNames[4]:   ['RKNE','RKNI'],
                  reducedMarkerNames[5]:   ['LKNE','LKNI'],
                  reducedMarkerNames[6]:   ['RFWT','RBWT'],
                  reducedMarkerNames[7]:   ['RFWT','LFWT','RBWT','LBWT'],
                  reducedMarkerNames[8]:   ['LFWT','LBWT'],
                  reducedMarkerNames[9]:   ['C6','T5','T10','BWT','STRN','CLAV','FWT'],
                  reducedMarkerNames[10]:  ['RPLM','RTHMB','RINDX','RMID','RPNKY'],
                  reducedMarkerNames[11]:  ['LPLM','LTHMB','LINDX','LMID','LPNKY'],
                  reducedMarkerNames[12]:  ['ROWR','RIWR'],
                  reducedMarkerNames[13]:  ['LOWR','LIWR'],
                  reducedMarkerNames[14]:  ['RELB','RIEL','RFRM'],
                  reducedMarkerNames[15]:  ['LELB','LIEL','LFRM'],
                  reducedMarkerNames[16]:  ['RSHO'],
                  reducedMarkerNames[17]:  ['RSHO','LSHO'],
                  reducedMarkerNames[18]:  ['LSHO'],
                  reducedMarkerNames[19]:  ['ARIEL','RFHD','LFHD','RBHD','LBHD']
                  }
    #   removing nan joints
    for nan_joint_name,_ in max_nans_per_column[::3].items():
        nan_joint_name = nan_joint_name[:-2]
        for markerFullList in markersMap.values():
            if nan_joint_name in markerFullList:
                markerFullList.remove(nan_joint_name)
                break
    # removing many contiguous zeros joints
    for zero_joint_name,contiguous_zeros in max_zeros_per_column[::3].items():
        if contiguous_zeros > 20:
            zero_joint_name = zero_joint_name[:-2]
            for markerFullList in markersMap.values():
                if zero_joint_name in markerFullList:
                    markerFullList.remove(zero_joint_name)
                    break
    originalDataColumns = [colname[:-2] for colname in list(data.iloc[:1,3::3].columns)]
    
    #   removing markers not present in the data markers
    for markerFullList in markersMap.values():
        for markerName in list(reversed(markerFullList)):
            if not markerName in originalDataColumns:
                markerFullList.remove(markerName)

    #   removing asymmetries across body
    for right_side_marker_indx,left_side_marker_indx in [(0,1),(2,3),(4,5),(6,8),(10,11),(12,13),(14,15)]: # indices of every marker on the left side and his symmetric
        markersLeft = markersMap[reducedMarkerNames[left_side_marker_indx]]
        markersRight = markersMap[reducedMarkerNames[right_side_marker_indx]]
        for elem in markersLeft:
            if not 'R'+elem[1:] in markersRight:
                markersLeft.remove(elem)
                if right_side_marker_indx == 6: 
                    markersMap[reducedMarkerNames[7]].remove(elem)
        for elem in markersRight:
            if not 'L'+elem[1:] in markersLeft:
                markersRight.remove(elem)
                if left_side_marker_indx == 8: 
                    markersMap[reducedMarkerNames[7]].remove(elem)

    for i,markerFullList in enumerate(markersMap.values()):
        if len(markerFullList) == 0:
            print(markersMap)
            raise Exception(f"can't reconstruct '{reducedMarkerNames[i]}' of 20 markers, because every marker of the full set is missing!!")
    return markersMap

In [15]:
reducedMarkerNames = ['left_foot',      # 0
                      'right_foot',     # 1          
                      'left_ank',       # 2      
                      'right_ank',      # 3                          
                      'left_knee',      # 4                          
                      'right_knee',     # 5                              
                      'left_hip',       # 6                          
                      'hip_central',    # 7                              
                      'right_hip',      # 8                          
                      'spine',          # 9                       
                      'left_hand',      # 10                           
                      'right_hand',     # 11                               
                      'left_wrist',     # 12                               
                      'right_wrist',    # 13                               
                      'left_elbow',     # 14                               
                      'right_elbow',    # 15                               
                      'left_shoulder',  # 16                               
                      'shoulder_center',# 17                                   
                      'right_shoulder', # 18                                   
                      'head']           # 19                       
fullMarkerNames = [colname[:-2] for colname in list(data.iloc[:1,3::3].columns)]
reduced20MarkersToFullMarkers = map_reduce_num_markers(reducedMarkerNames)
reduced20MarkersToFullMarkersX = {key+'_X': [elem+' X' for elem in reduced20MarkersToFullMarkers[key]] for key in reduced20MarkersToFullMarkers.keys()}
reduced20MarkersToFullMarkersY = {key+'_Y': [elem+' Y' for elem in reduced20MarkersToFullMarkers[key]] for key in reduced20MarkersToFullMarkers.keys()}
reduced20MarkersToFullMarkersZ = {key+'_Z': [elem+' Z' for elem in reduced20MarkersToFullMarkers[key]] for key in reduced20MarkersToFullMarkers.keys()}

posTable = data.iloc[:,3:]

posTableX = posTable.iloc[:,::3]
posTableY = posTable.iloc[:,1::3]
posTableZ = posTable.iloc[:,2::3]

reducedPosTableX = pd.concat([posTableX[reduced20MarkersToFullMarkersX[colName+'_X']].mean(axis=1) for colName in reducedMarkerNames],axis=1,keys=[name+'_X' for name in reducedMarkerNames])
reducedPosTableY = pd.concat([posTableY[reduced20MarkersToFullMarkersY[colName+'_Y']].mean(axis=1) for colName in reducedMarkerNames],axis=1,keys=[name+'_Y' for name in reducedMarkerNames])
reducedPosTableZ = pd.concat([posTableZ[reduced20MarkersToFullMarkersZ[colName+'_Z']].mean(axis=1) for colName in reducedMarkerNames],axis=1,keys=[name+'_Z' for name in reducedMarkerNames])

#reducedPosTableX = (reducedPosTableX - reducedPosTableX.min(axis=None))/(reducedPosTableX.max(axis=None)-reducedPosTableX.min(axis=None))
#reducedPosTableY = (reducedPosTableY - reducedPosTableY.min(axis=None))/(reducedPosTableY.max(axis=None)-reducedPosTableY.min(axis=None))
#reducedPosTableZ = (reducedPosTableZ - reducedPosTableZ.min(axis=None))/(reducedPosTableZ.max(axis=None)-reducedPosTableZ.min(axis=None))

def xyz_tables_to_xyz_columns(tablesList):
    xTable,yTable,zTable = tablesList
    mergedTable = pd.DataFrame()
    for joint in range(xTable.shape[1]):
        mergedTable = pd.concat([mergedTable,xTable.iloc[:,joint],yTable.iloc[:,joint],zTable.iloc[:,joint]],axis=1)
    return mergedTable

reducedPosTable = xyz_tables_to_xyz_columns([reducedPosTableX,reducedPosTableY,reducedPosTableZ])
reducedPosTable = (reducedPosTable - reducedPosTable.min(axis=None))/(reducedPosTable.max(axis=None)-reducedPosTable.min(axis=None))
reducedPosTable.head()

,left_foot_X,left_foot_Y,left_foot_Z,right_foot_X,right_foot_Y,right_foot_Z,left_ank_X,left_ank_Y,left_ank_Z,right_ank_X,...,left_shoulder_Z,shoulder_center_X,shoulder_center_Y,shoulder_center_Z,right_shoulder_X,right_shoulder_Y,right_shoulder_Z,head_X,head_Y,head_Z
0,0.442782,0.431855,0.272077,0.350959,0.432065,0.266551,0.445094,0.442759,0.303456,0.348661,...,0.264937,0.406429,0.826919,0.265189,0.355623,0.828596,0.265440,0.408212,0.876711,0.202353
1,0.442789,0.431857,0.272071,0.350919,0.432035,0.266578,0.445034,0.442795,0.303494,0.348622,...,0.265126,0.406000,0.826985,0.265180,0.355200,0.828607,0.265233,0.407979,0.876343,0.202080
2,0.442785,0.431858,0.272076,0.350863,0.432007,0.266608,0.444968,0.442803,0.303538,0.348522,...,0.265281,0.405640,0.827014,0.265155,0.354854,0.828586,0.265029,0.407745,0.875934,0.201762
3,0.442775,0.431857,0.272075,0.350801,0.431982,0.266635,0.444879,0.442833,0.303602,0.348389,...,0.265420,0.405292,0.827038,0.265150,0.354524,0.828579,0.264880,0.407510,0.875475,0.201422
4,0.442773,0.431859,0.272085,0.350742,0.431960,0.266662,0.444816,0.442837,0.303656,0.348258,...,0.265521,0.404957,0.827046,0.265126,0.354186,0.828555,0.264731,0.407258,0.874988,0.201055


## Writing to files final clean

In [16]:
import os

PROCESSED_PATH = 'data/reprocessed/'

dest_dir = os.path.join(PROCESSED_PATH,folder)
if not os.path.exists(dest_dir): 
    os.mkdir(dest_dir)
dest_file_path = os.path.join(dest_dir,trial+'_frag'+fragId+'.csv')
if not os.path.isfile(dest_file_path) or True:
    reducedPosTable.to_csv(dest_file_path,index=False)

## Plotting reduced markers

In [28]:
from matplotlib import pyplot as plt
%matplotlib
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
if sampleAnno < 45:
    posTableY = reducedPosTable.iloc[:,::3]
    posTableZ = reducedPosTable.iloc[:,1::3]
    posTableX = reducedPosTable.iloc[:,2::3]
else:
    posTableX = reducedPosTable.iloc[:,::3]
    posTableY = reducedPosTable.iloc[:,1::3]
    posTableZ = reducedPosTable.iloc[:,2::3]

jointsFrom = [1, 3, 5, 7, 2, 4, 6, 9, 8,10,11,13,15,17,12,14,16,19,18]
jointsTo =   [3, 5, 7, 8, 4, 6, 9, 8,10,18,13,15,17,18,14,16,19,18,20]
jointsFrom = list(map(lambda x: x-1,jointsFrom))
jointsTo = list(map(lambda x: x-1,jointsTo))
edges = np.array(list(zip(jointsFrom,jointsTo))+list(zip(jointsTo,jointsFrom)))
adjacencyMatrix = np.zeros((20,20),dtype=bool)
adjacencyMatrix[edges[:,0],edges[:,1]] = True

plt.close("all")
fig = plt.figure()
ax:plt.Axes = fig.add_subplot(111, projection='3d')

minMax = np.zeros((2,3))
minMax[0,:] = [np.nanmin(posTableX.values),np.nanmin(posTableY.values),np.nanmin(posTableZ.values)]
minMax[1,:] = [np.nanmax(posTableX.values),np.nanmax(posTableY.values),np.nanmax(posTableZ.values)]

# Set the window title
fig.canvas.manager.window.title("3D Movement\t(Scroll with mouse wheel)")

# Set the initial time index
time_index = 0

# Function to update the plot based on the slider value
def update_plot(val):
    ax.cla()  # Clear the previous plot
    
    # Filter the data based on the current time index
    filteredX = posTableX.iloc[val]
    filteredY = posTableY.iloc[val]
    filteredZ = posTableZ.iloc[val]
    
    ax.scatter(filteredX,filteredY,filteredZ)

    # Add edges based on the weight matrix
    for i in range(20):
        for j in range(i + 1, 20):
            if adjacencyMatrix[i,j]:
                ax.plot([filteredX[i], filteredX[j]],
                        [filteredY[i], filteredY[j]],
                        [filteredZ[i], filteredZ[j]],
                        color='gray', linestyle='-', linewidth=1)

    ax.set_xlim([-0.1,0.8])#minMax[0,0],minMax[1,0]])
    ax.set_ylim([-0.1,0.8])#minMax[0,1],minMax[1,1]])
    ax.set_zlim([minMax[0,2],minMax[1,2]])

    ax.set_xlabel('X', fontsize=12)
    ax.set_ylabel('Y', fontsize=12)
    ax.set_zlabel('Z', fontsize=12)
    ax.set_title("Movement "+str(sampleAnno))

    fig.canvas.draw_idle()

# Create a slider widget
slider_ax = plt.axes([0.2, 0.03, 0.6, 0.03])
maxValue = posTable.shape[0]-1
slider = Slider(slider_ax, 'TimeIndex:', 0, maxValue, valinit=time_index, valstep=1)


# Define a function to update the slider value with the mouse wheel
def on_scroll(event):
    if event.button == 'down':
        if slider.val + slider.valstep*2 <= maxValue:
            slider.set_val(slider.val + slider.valstep*2)
    elif event.button == 'up':
        if slider.val - slider.valstep*2 >= 0:
            slider.set_val(slider.val - slider.valstep*2)
        

# Connect the mouse wheel event to the function
fig.canvas.mpl_connect('scroll_event', on_scroll)


# Register the update_plot function with the slider widget
slider.on_changed(update_plot)

# Initial plot
update_plot(time_index)

# Show the plot
plt.show()

Using matplotlib backend: TkAgg


## Plotting full markers

In [51]:
from matplotlib import pyplot as plt
%matplotlib
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
if sampleAnno < 45:
    posTableY = posTable.iloc[:,::3]
    posTableZ = posTable.iloc[:,1::3]
    posTableX = posTable.iloc[:,2::3]
else:
    posTableX = posTable.iloc[:,::3]
    posTableY = posTable.iloc[:,1::3]
    posTableZ = posTable.iloc[:,2::3]
    

plt.close("all")
fig = plt.figure()
ax:plt.Axes = fig.add_subplot(111, projection='3d')

minMax = np.zeros((2,3))
minMax[0,:] = [np.nanmin(posTableX.values),np.nanmin(posTableY.values),np.nanmin(posTableZ.values)]
minMax[1,:] = [np.nanmax(posTableX.values),np.nanmax(posTableY.values),np.nanmax(posTableZ.values)]

# Set the window title
fig.canvas.manager.window.title("3D Movement\t(Scroll with mouse wheel)")

# Set the initial time index
time_index = 0
column_names = [ col[:-2] for col in list(posTable.columns)[::3] ]

# Function to update the plot based on the slider value
def update_plot(val):
    ax.cla()  # Clear the previous plot
    
    # Filter the data based on the current time index
    filteredX = posTableX.iloc[val]
    filteredY = posTableY.iloc[val]
    filteredZ = posTableZ.iloc[val]
    
    ax.scatter(filteredX,filteredY,filteredZ)

    for i, name in enumerate(column_names):
        ax.text(filteredX[i], filteredY[i], filteredZ[i], name, fontsize=8, ha='left')


    ax.set_xlim([minMax[0,0],minMax[1,0]])
    ax.set_ylim([minMax[0,1],minMax[1,1]])
    ax.set_zlim([minMax[0,2],minMax[1,2]])

    ax.set_xlabel('X', fontsize=12)
    ax.set_ylabel('Y', fontsize=12)
    ax.set_zlabel('Z', fontsize=12)
    #ax.set_title("Movement "+str(picked))

    fig.canvas.draw_idle()

# Create a slider widget
slider_ax = plt.axes([0.2, 0.03, 0.6, 0.03])
maxValue = posTable.shape[0]-1
slider = Slider(slider_ax, 'TimeIndex:', 0, maxValue, valinit=time_index, valstep=1)


# Define a function to update the slider value with the mouse wheel
def on_scroll(event):
    if event.button == 'down':
        if slider.val + slider.valstep*2 <= maxValue:
            slider.set_val(slider.val + slider.valstep*2)
    elif event.button == 'up':
        if slider.val - slider.valstep*2 >= 0:
            slider.set_val(slider.val - slider.valstep*2)
        

# Connect the mouse wheel event to the function
fig.canvas.mpl_connect('scroll_event', on_scroll)


# Register the update_plot function with the slider widget
slider.on_changed(update_plot)

# Initial plot
update_plot(time_index)

# Show the plot
plt.show()


Using matplotlib backend: TkAgg


In [19]:
from matplotlib import pyplot as plt
%matplotlib
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
import numpy as np


plt.close("all")
fig = plt.figure()
ax:plt.Axes = fig.add_subplot(111, projection='3d')

minMax = np.zeros((2,3))
minMax[0,:] = [np.nanmin(posTableX.values),np.nanmin(posTableY.values),np.nanmin(posTableZ.values)]
minMax[1,:] = [np.nanmax(posTableX.values),np.nanmax(posTableY.values),np.nanmax(posTableZ.values)]

minMax[0,:] = [np.nanmin(posTableX.iloc[:1,:].values),np.nanmin(posTableY.iloc[:1,:].values),np.nanmin(posTableZ.iloc[:1,:].values)]
minMax[1,:] = [np.nanmax(posTableX.iloc[:1,:].values),np.nanmax(posTableY.iloc[:1,:].values),np.nanmax(posTableZ.iloc[:1,:].values)]

# Set the window title
fig.canvas.manager.window.title("3D Movement\t(Scroll with mouse wheel)")

# Set the initial time index
time_index = 0

# Function to update the plot based on the slider value
def update_plot(val):
    ax.cla()  # Clear the previous plot
    
    # Filter the data based on the current time index
    filteredX = posTableX.iloc[val]
    filteredY = posTableY.iloc[val]
    filteredZ = posTableZ.iloc[val]
    
    ax.scatter(filteredX,filteredY,filteredZ,s=0)
    
    for i, col_name in enumerate(posTableX.columns):
        ax.text(filteredX[i], filteredY[i], filteredZ[i], col_name[:-2], horizontalalignment='center',fontsize=6, color='black')

    ax.set_xticks([minMax[0,0],minMax[1,0]])
    ax.set_yticks([minMax[0,1],minMax[1,1]])
    ax.set_zticks([minMax[0,2],minMax[1,2]])
    ax.set_xlim([minMax[0,0],minMax[1,0]])
    ax.set_ylim([minMax[0,1],minMax[1,1]])
    ax.set_zlim([minMax[0,2],minMax[1,2]])
    
    fig.canvas.draw_idle()

# Create a slider widget
slider_ax = plt.axes([0.2, 0.03, 0.6, 0.03])
maxValue = posTable.shape[0]-1
slider = Slider(slider_ax, 'TimeIndex:', 0, maxValue, valinit=time_index, valstep=1)


# Define a function to update the slider value with the mouse wheel
def on_scroll(event):
    if event.button == 'down':
        if slider.val + slider.valstep*2 <= maxValue:
            slider.set_val(slider.val + slider.valstep*2)
    elif event.button == 'up':
        if slider.val - slider.valstep*2 >= 0:
            slider.set_val(slider.val - slider.valstep*2)
        

# Connect the mouse wheel event to the function
fig.canvas.mpl_connect('scroll_event', on_scroll)


# Register the update_plot function with the slider widget
slider.on_changed(update_plot)

# Initial plot
update_plot(time_index)

# Show the plot
plt.show()


Using matplotlib backend: TkAgg


In [242]:
plt.close('all')

# All in One

In [195]:
from matplotlib import pyplot as plt
%matplotlib
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

def visualize_a_sample(posTable, sample):
    posTableX = posTable.iloc[:,::3]
    posTableY = posTable.iloc[:,2::3]
    posTableZ = posTable.iloc[:,1::3]
    jointsFrom = [1, 3, 5, 7, 2, 4, 6, 9, 8,10,11,13,15,17,12,14,16,19,18]
    jointsTo =   [3, 5, 7, 8, 4, 6, 9, 8,10,18,13,15,17,18,14,16,19,18,20]
    jointsFrom = list(map(lambda x: x-1,jointsFrom))
    jointsTo = list(map(lambda x: x-1,jointsTo))
    edges = np.array(list(zip(jointsFrom,jointsTo))+list(zip(jointsTo,jointsFrom)))
    adjacencyMatrix = np.zeros((20,20),dtype=bool)
    adjacencyMatrix[edges[:,0],edges[:,1]] = True

    plt.close('all')
    fig = plt.figure(figsize=(10,8))
    ax:plt.Axes = fig.add_subplot(111, projection='3d')
    ax.view_init(elev=0, azim=0)
    fig.canvas.manager.full_screen_toggle()

    
    minMax = np.zeros((2,3))
    minMax[0,:] = [posTableX.values.min(),posTableY.values.min(),posTableZ.values.min()]
    minMax[1,:] = [posTableX.values.max(),posTableY.values.max(),posTableZ.values.max()]

    # Set appropriate axis limits
    ax.set_xlim([minMax[0,0],minMax[1,0]])
    ax.set_ylim([0,1])
    ax.set_zlim([minMax[0,2],minMax[1,2]])

    # Set the window title
    fig.canvas.manager.window.title("3D Movement\t(Scroll with mouse wheel)")

    # Set the initial time index
    time_index = 0
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'yellow', 'cyan']

    # Function to update the plot based on the slider value
    def update_plot(val):
        ax.cla()  # Clear the previous plot

        # Filter the data based on the current time index
        filteredX = posTableX.iloc[val]
        filteredY = posTableY.iloc[val]
        filteredZ = posTableZ.iloc[val]

        ax.scatter(filteredX,filteredY,filteredZ)

        # Add edges based on the weight matrix
        for i in range(20):
            for j in range(i + 1, 20):
                if adjacencyMatrix[i,j]:
                    ax.plot([filteredX[i], filteredX[j]],
                            [filteredY[i], filteredY[j]],
                            [filteredZ[i], filteredZ[j]],
                            color='gray', linestyle='-', linewidth=1)


        ax.set_xlim([minMax[0,0],minMax[1,0]])
        ax.set_ylim([0,1])
        ax.set_zlim([minMax[0,2],minMax[1,2]])

        ax.set_xlabel('X', fontsize=12)
        ax.set_ylabel('Y', fontsize=12)
        ax.set_zlabel('Z', fontsize=12)
        ax.set_title("Movement "+str(sample))

        fig.canvas.draw_idle()

    # Create a slider widget
    slider_ax = plt.axes([0.2, 0.03, 0.6, 0.03])
    maxValue = posTableX.shape[0]-1
    slider = Slider(slider_ax, 'TimeIndex:', 0, maxValue, valinit=time_index, valstep=1)


    # Define a function to update the slider value with the mouse wheel
    def on_scroll(event):
        if event.button == 'down':
            if slider.val + slider.valstep <= maxValue:
                slider.set_val(slider.val + slider.valstep)
        elif event.button == 'up':
            if slider.val - slider.valstep >= 0:
                slider.set_val(slider.val - slider.valstep)


    # Connect the mouse wheel event to the function
    fig.canvas.mpl_connect('scroll_event', on_scroll)


    # Register the update_plot function with the slider widget
    slider.on_changed(update_plot)

    # Initial plot
    update_plot(time_index)

    # Show the plot
    plt.show(block=True)

# Function to find the maximum consecutive zeros values from a mask series
def max_consecutive_zeros(series):
        consecutive_true_count = 0
        max_consecutive_true_count = 0

        for value in series:
            if value:
                consecutive_true_count += 1
                if consecutive_true_count > max_consecutive_true_count:
                    max_consecutive_true_count = consecutive_true_count
            else:
                consecutive_true_count = 0

        return max_consecutive_true_count


Using matplotlib backend: TkAgg


In [206]:
import csv
import os
import pandas as pd
from tqdm.notebook import tqdm
from itertools import groupby

def custom_header_reader(file:'TextIOWrapper'):
    csvReader = csv.reader(file,delimiter='\t')
    for row, text in enumerate(csvReader):
        if row == 2: numMarkers = int(text[-1])
        elif row == 10: 
            columnNames = text[:-1]
            if columnNames[0] == 'Frame': 
                break
        elif row == 11: columnNames = text[:-1]; break
    return numMarkers, columnNames, csvReader

def line_reader(csvReader,fromSecond,toSecond):
    for line in csvReader:
        if fromSecond <= float(line[1]) <= toSecond:
            yield line[:len(columnNames)]
        elif float(line[1]) > toSecond:
            break


ANNOTATIONS_PATH = 'data'
RAW_PATH = ANNOTATIONS_PATH+'/raw'
#with open(ANNOTATIONS_PATH+'/annotations.txt','r') as file:
#    annotations = file.read().splitlines()
with open(ANNOTATIONS_PATH+'/annotationsVSingle.txt','r') as file:
    annotations = file.read().splitlines()[1:]#[line for line in file.read().splitlines() if 'no val' in line]
    #print(len(annotations))
    
for sampleAnno in tqdm(range(len(annotations)),desc="Compressing markers and saving to csv reprocessed"):
    if '(A)' in annotations[sampleAnno] or '(B)' in annotations[sampleAnno]:
        folder, trial, fragId, side, OoM, startSec, endSec = annotations[sampleAnno].replace(' ','').split(',')[:7]
    else:
        folder, trial, fragId, OoM, startSec, endSec = annotations[sampleAnno].replace(' ','').split(',')[:6]
    sampleData = os.path.join(RAW_PATH,folder,trial+'.tsv')
    with open(sampleData,'r') as file:
        numMarkers, columnNames, readerBuffer = custom_header_reader(file)
        data = pd.DataFrame(line_reader(readerBuffer,float(startSec),float(endSec)),columns=columnNames).astype(dict(zip(columnNames,[int,float,str]+[float]*len(columnNames[3:]))))

    if data['Time'].iloc[0] > float(startSec) or data['Time'].iloc[-1] < float(endSec):
        raise Exception(f'\x1b[31mNOT FULL FRAMES!!! data starts at: {data["Time"].iloc[0]} of {float(startSec)} and ends at {data["Time"].iloc[-1]} of {float(endSec)}\x1b[0m')
    else:
        pass


    if '_A' in data.columns[3] or '_B' in data.columns[3]:
        data = data[['Frame','Time','SMPTE']+data.filter(like=f'_{side[1]}').columns.tolist()]
        data.columns = [col.replace(f'_{side[1]}','') for col in data.columns]
    elif '61A' in data.columns[3] or '61B' in data.columns[3]:
        data = data[['Frame','Time','SMPTE']+data.filter(like=f'61{side[1]}').columns.tolist()]
        data.columns = [col.replace(f'61{side[1]}_','') for col in data.columns]
    if '_01' in data.columns[3]:
        data.columns = [column for column in data.columns[:3]]+[column[:-5]+column[-2:] for column in data.columns[3:]]
    if 'HeadFront' in data.columns[3] or 'RKneeIn' in data.columns[3]:
        mappings = pd.read_csv("data/raw/Sport42MarkersTo64.csv")
        mappings = mappings.set_index(mappings.columns[0])[mappings.columns[1]].to_dict()
        data.columns = [mappings[col[:-2]]+col[-2:] if col[:-2] in mappings.keys() else col for col in data.columns]

    nans = data[data.columns[(data.isna().sum(axis=0) > 0 ).to_list()]].isna().sum(axis=0).sort_values(ascending=False)



    max_nans_per_column = data.apply(lambda col: max(len(list(group))*value for value, group in groupby(col.isna().tolist())))

    max_nans_per_column = max_nans_per_column[max_nans_per_column > 0].sort_values(ascending=False)


    zeros = data[data.columns[(data.eq(0.0).sum(axis=0) > 0).to_list()]].eq(0.0).sum(axis=0).sort_values(ascending=False)

    # THIS IS A FIX FOR MISSING QTM AT DATA 58-59
    if len(zeros.index) > 15:
        for name_col,num_zeros in zeros.items():
            if num_zeros < 50:
                data[name_col] = data[name_col].replace(0.0,np.nan).interpolate(method='spline', order=3)
    if sampleAnno == 1:
        for name_col,num_zeros in zeros.items():
            if num_zeros < 10:
                data[name_col] = data[name_col].replace(0.0,np.nan).interpolate(method='spline', order=3)
    #for name_col,num_zeros in zeros.items():
    #    if num_zeros < 20:
    #        data[name_col] = data[name_col].replace(0.0,np.nan).interpolate(method='spline', order=3)

    zeros = data[data.columns[(data.eq(0.0).sum(axis=0) > 0).to_list()]].eq(0.0).sum(axis=0).sort_values(ascending=False)

    max_zeros_per_column = data.apply(lambda col: max(len(list(group))*value for value, group in groupby(col.eq(0.0).tolist())))
    max_zeros_per_column = max_zeros_per_column[max_zeros_per_column > 0].sort_values(ascending=False)

    def map_reduce_num_markers(reducedMarkerNames:list):
        markersMap = {reducedMarkerNames[0]:   ['RHEL','RMT1','RMT5','RTOE'],
                      reducedMarkerNames[1]:   ['LHEL','LMT1','LMT5','LTOE'],
                      reducedMarkerNames[2]:   ['RANK'],
                      reducedMarkerNames[3]:   ['LANK'],
                      reducedMarkerNames[4]:   ['RKNE','RKNI'],#,'RSHN','RShinFrontHigh'],
                      reducedMarkerNames[5]:   ['LKNE','LKNI'],#,'LSHN','LShinFrontHigh'],
                      reducedMarkerNames[6]:   ['RFWT','RBWT'],
                      reducedMarkerNames[7]:   ['RFWT','LFWT','RBWT','LBWT'],
                      reducedMarkerNames[8]:   ['LFWT','LBWT'],
                      reducedMarkerNames[9]:   ['C6','C7','T5','T10','BWT','STRN','CLAV','FWT'],
                      reducedMarkerNames[10]:  ['RPLM','RTHMB','RINDX','RMID','RPNKY'],
                      reducedMarkerNames[11]:  ['LPLM','LTHMB','LINDX','LMID','LPNKY'],
                      reducedMarkerNames[12]:  ['ROWR','RIWR'],
                      reducedMarkerNames[13]:  ['LOWR','LIWR'],
                      reducedMarkerNames[14]:  ['RELB','RIEL','RFRM'],
                      reducedMarkerNames[15]:  ['LELB','LIEL','LFRM'],
                      reducedMarkerNames[16]:  ['RSHO'],
                      reducedMarkerNames[17]:  ['RSHO','LSHO'],#'C7'],
                      reducedMarkerNames[18]:  ['LSHO'],
                      reducedMarkerNames[19]:  ['ARIEL','RFHD','LFHD','RBHD','LBHD']
                      }
        
        # removing nan joints
        for nan_joint_name,_ in max_nans_per_column[::3].items():
            nan_joint_name = nan_joint_name[:-2]
            for markerFullList in markersMap.values():
                if nan_joint_name in markerFullList:
                    markerFullList.remove(nan_joint_name)
                    break

        # removing many contiguous zeros joints
        for zero_joint_name,contiguous_zeros in max_zeros_per_column[::3].items():
            if contiguous_zeros > 20:
                zero_joint_name = zero_joint_name[:-2]
                for markerFullList in markersMap.values():
                    if zero_joint_name in markerFullList:
                        markerFullList.remove(zero_joint_name)
                        break
        originalDataColumns = [colname[:-2] for colname in list(data.iloc[:1,3::3].columns)]

        # removing markers not present in the data markers
        for markerFullList in markersMap.values():
            for markerName in list(reversed(markerFullList)):
                if not markerName in originalDataColumns:
                    markerFullList.remove(markerName)

        #   removing asymmetries across body
        for right_side_marker_indx,left_side_marker_indx in [(0,1),(2,3),(4,5),(6,8),(10,11),(12,13),(14,15)]: # indices of every marker on the left side and his symmetric
            markersLeft = markersMap[reducedMarkerNames[left_side_marker_indx]]
            markersRight = markersMap[reducedMarkerNames[right_side_marker_indx]]
            for elem in markersLeft:
                if not 'R'+elem[1:] in markersRight:
                    markersLeft.remove(elem)
                    if right_side_marker_indx == 6: 
                        markersMap[reducedMarkerNames[7]].remove(elem)
            for elem in markersRight:
                if not 'L'+elem[1:] in markersLeft:
                    markersRight.remove(elem)
                    if left_side_marker_indx == 8: 
                        markersMap[reducedMarkerNames[7]].remove(elem)

        for i,markerFullList in enumerate(markersMap.values()):
            if len(markerFullList) == 0:
                print(markersMap)
                raise Exception(f"can't reconstruct '{reducedMarkerNames[i]}' of 20 markers, because every marker of the full set is missing!!")
        return markersMap

    reducedMarkerNames = ['right_foot',      # 1
                          'left_foot',       # 2        
                          'right_ank',       # 3      
                          'left_ank',        # 4                          
                          'right_knee',      # 5                          
                          'left_knee',       # 6                              
                          'right_hip',       # 7                          
                          'hip_central',     # 8                              
                          'left_hip',        # 9                          
                          'spine',           # 10                       
                          'right_hand',      # 11                           
                          'left_hand',       # 12                               
                          'right_wrist',     # 13                               
                          'left_wrist',      # 14                               
                          'right_elbow',     # 15                               
                          'left_elbow',      # 16                               
                          'right_shoulder',  # 17                               
                          'shoulder_center', # 18                                   
                          'left_shoulder',   # 19                                   
                          'head']            # 20                       
    fullMarkerNames = [colname[:-2] for colname in list(data.iloc[:1,3::3].columns)]
    reduced20MarkersToFullMarkers = map_reduce_num_markers(reducedMarkerNames)
    reduced20MarkersToFullMarkersX = {key+'_X': [elem+' X' for elem in reduced20MarkersToFullMarkers[key]] for key in reduced20MarkersToFullMarkers.keys()}
    reduced20MarkersToFullMarkersY = {key+'_Y': [elem+' Y' for elem in reduced20MarkersToFullMarkers[key]] for key in reduced20MarkersToFullMarkers.keys()}
    reduced20MarkersToFullMarkersZ = {key+'_Z': [elem+' Z' for elem in reduced20MarkersToFullMarkers[key]] for key in reduced20MarkersToFullMarkers.keys()}

    posTable = data.iloc[:,3:]

    posTableX = posTable.iloc[:,::3]
    posTableY = posTable.iloc[:,1::3]
    posTableZ = posTable.iloc[:,2::3]

    reducedPosTableX = pd.concat([posTableX[reduced20MarkersToFullMarkersX[colName+'_X']].mean(axis=1) for colName in reducedMarkerNames],axis=1,keys=[name+'_X' for name in reducedMarkerNames])
    reducedPosTableY = pd.concat([posTableY[reduced20MarkersToFullMarkersY[colName+'_Y']].mean(axis=1) for colName in reducedMarkerNames],axis=1,keys=[name+'_Y' for name in reducedMarkerNames])
    reducedPosTableZ = pd.concat([posTableZ[reduced20MarkersToFullMarkersZ[colName+'_Z']].mean(axis=1) for colName in reducedMarkerNames],axis=1,keys=[name+'_Z' for name in reducedMarkerNames])

    def xyz_tables_to_xyz_columns(tablesList):
        xTable,yTable,zTable = tablesList
        mergedTable = pd.DataFrame()
        for joint in range(xTable.shape[1]):
            mergedTable = pd.concat([mergedTable,xTable.iloc[:,joint],yTable.iloc[:,joint],zTable.iloc[:,joint]],axis=1)
        return mergedTable

    reducedPosTable = xyz_tables_to_xyz_columns([reducedPosTableX,reducedPosTableY,reducedPosTableZ])
    reducedPosTable = (reducedPosTable - reducedPosTable.min(axis=None))/(reducedPosTable.max(axis=None)-reducedPosTable.min(axis=None))
    #reducedPosTable = reducedPosTable.rolling(window=2).mean().fillna(method='bfill')
    
    if reducedPosTable.eq(0.0).apply(max_consecutive_zeros).max() > 1:
        print(sampleAnno, reducedPosTable.eq(0.0).apply(max_consecutive_zeros).max())
        visualize_a_sample(reducedPosTable,sampleAnno)

    PROCESSED_PATH = 'data/reprocessed/'

    dest_dir = os.path.join(PROCESSED_PATH,folder)
    if not os.path.exists(dest_dir): 
        os.mkdir(dest_dir)
    dest_file_path = os.path.join(dest_dir,trial+'_frag'+fragId+'.csv')
    if not os.path.isfile(dest_file_path) or True:
        reducedPosTable.to_csv(dest_file_path,index=False)

Compressing markers and saving to csv reprocessed:   0%|          | 0/60 [00:00<?, ?it/s]